In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import csv
import json

In [2]:
r = requests.get('https://data.melbourne.vic.gov.au/resource/4n3a-s6rn.json')
json_data = json.loads(r.text)

In [3]:
json_data

[{'deviceid': '17571',
  'arrivaltime': '2020-02-26T10:47:05.000',
  'departuretime': '2020-02-26T10:47:42.000',
  'durationminutes': '0',
  'streetmarker': '13720E',
  'signplateid': '196',
  'sign': '2P MTR M-SAT 7:30-18:30',
  'areaname': 'Docklands',
  'streetid': '1097',
  'streetname': 'PEARL RIVER ROAD',
  'betweenstreet1id': '590',
  'betweenstreet1': 'DOCKLANDS DRIVE',
  'betweenstreet2id': '1066',
  'betweenstreet2': 'OBSERVATION DRIVE',
  'sideofstreet': '2',
  'sidename': 'East',
  'bayid': '6503',
  'inviolation': '0',
  'vehiclepresent': '0'},
 {'deviceid': '17571',
  'arrivaltime': '2020-02-26T18:30:00.000',
  'departuretime': '2020-02-27T00:00:00.000',
  'durationminutes': '330',
  'streetmarker': '13720E',
  'areaname': 'Docklands',
  'streetid': '1097',
  'streetname': 'PEARL RIVER ROAD',
  'betweenstreet1id': '590',
  'betweenstreet1': 'DOCKLANDS DRIVE',
  'betweenstreet2id': '1066',
  'betweenstreet2': 'OBSERVATION DRIVE',
  'sideofstreet': '2',
  'sidename': 'East'

In [4]:
df = json_normalize(resp.json())
df.head()

NameError: name 'json_normalize' is not defined

In [5]:
url = "https://data.melbourne.vic.gov.au/resource/4n3a-s6rn.json"
historic_df = pd.read_json(url)
historic_df.head()

,deviceid,arrivaltime,departuretime,durationminutes,streetmarker,signplateid,sign,areaname,streetid,streetname,betweenstreet1id,betweenstreet1,betweenstreet2id,betweenstreet2,sideofstreet,sidename,bayid,inviolation,vehiclepresent
0,17605,2020-01-31T15:08:32.000,2020-01-31T16:22:54.000,74,13762E,550.0,Loading Zone 60mins,Docklands,592,DOEPEL WAY,590,DOCKLANDS DRIVE,1048,NEWQUAY PROMENADE,2,East,6650,0,0
1,17585,2020-02-26T05:43:24.000,2020-02-26T06:52:06.000,69,13740E,NaN,NaN,Docklands,1097,PEARL RIVER ROAD,590,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6513,0,1
2,17562,2020-01-13T22:55:52.000,2020-01-13T22:56:09.000,1,13711W,NaN,NaN,Docklands,1405,WATTLE ROAD,880,LEVEN AVENUE,590,DOCKLANDS DRIVE,5,West,6483,0,1
3,17554,2020-02-06T00:00:00.000,2020-02-06T07:30:00.000,450,13697W,NaN,NaN,Docklands,1043,NAVIGATION DRIVE,528,COLLINS STREET,123,BOURKE STREET,5,West,6435,0,0
4,17580,2020-01-08T17:05:43.000,2020-01-08T17:40:29.000,35,13730E,196.0,2P MTR M-SAT 7:30-18:30,Docklands,1097,PEARL RIVER ROAD,590,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6508,0,0


In [12]:
historic_df['bayid'] = historic_df['bayid'].astype(int)

In [14]:
historic_df.rename(columns = {'bayid':'bay_id'}, inplace = True) 
historic_df

,deviceid,arrivaltime,departuretime,durationminutes,streetmarker,signplateid,sign,areaname,streetid,streetname,betweenstreet1id,betweenstreet1,betweenstreet2id,betweenstreet2,sideofstreet,sidename,bay_id,inviolation,vehiclepresent
0,17605,2020-01-31T15:08:32.000,2020-01-31T16:22:54.000,74,13762E,550.0,Loading Zone 60mins,Docklands,592,DOEPEL WAY,590,DOCKLANDS DRIVE,1048,NEWQUAY PROMENADE,2,East,6650,0,0
1,17585,2020-02-26T05:43:24.000,2020-02-26T06:52:06.000,69,13740E,NaN,NaN,Docklands,1097,PEARL RIVER ROAD,590,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6513,0,1
2,17562,2020-01-13T22:55:52.000,2020-01-13T22:56:09.000,1,13711W,NaN,NaN,Docklands,1405,WATTLE ROAD,880,LEVEN AVENUE,590,DOCKLANDS DRIVE,5,West,6483,0,1
3,17554,2020-02-06T00:00:00.000,2020-02-06T07:30:00.000,450,13697W,NaN,NaN,Docklands,1043,NAVIGATION DRIVE,528,COLLINS STREET,123,BOURKE STREET,5,West,6435,0,0
4,17580,2020-01-08T17:05:43.000,2020-01-08T17:40:29.000,35,13730E,196.0,2P MTR M-SAT 7:30-18:30,Docklands,1097,PEARL RIVER ROAD,590,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6508,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,17599,2020-02-19T10:06:27.000,2020-02-19T10:15:48.000,9,13756E,287.0,1P MTR M-SAT 7:30-18:30,Docklands,1097,PEARL RIVER ROAD,590,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6521,0,1
996,17584,2020-02-25T09:48:20.000,2020-02-25T14:13:03.000,265,13738E,196.0,2P MTR M-SAT 7:30-18:30,Docklands,1097,PEARL RIVER ROAD,590,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6512,0,0
997,17599,2020-01-22T10:21:58.000,2020-01-22T10:33:06.000,12,13756E,287.0,1P MTR M-SAT 7:30-18:30,Docklands,1097,PEARL RIVER ROAD,590,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6521,0,0
998,17642,2020-01-09T11:35:14.000,2020-01-09T11:39:26.000,4,13808N,126.0,1/4P M-SUN 7:30-18:30,Docklands,590,DOCKLANDS DRIVE,1097,PEARL RIVER ROAD,1398,WATERFRONT WAY,3,North,6474,0,0


In [6]:
url = "https://data.melbourne.vic.gov.au/resource/vh2v-4nfs.json"
live_df = pd.read_json(url)
live_df

,bay_id,st_marker_id,status,location,lat,lon,:@computed_region_evbi_jbp8
0,2242,3930N,Unoccupied,"{'latitude': '-37.81219457073095', 'longitude'...",-37.812195,144.954629,1
1,3843,C5566,Unoccupied,"{'latitude': '-37.80641793309145', 'longitude'...",-37.806418,144.953969,1
2,2569,C3128,Present,"{'latitude': '-37.81353464672647', 'longitude'...",-37.813535,144.957970,1
3,4043,C6124,Unoccupied,"{'latitude': '-37.80451697889771', 'longitude'...",-37.804517,144.952376,1
4,4344,C7042,Present,"{'latitude': '-37.80749831220392', 'longitude'...",-37.807498,144.958440,1
...,...,...,...,...,...,...,...
995,3316,6000N,Present,"{'latitude': '-37.80983394631515', 'longitude'...",-37.809834,144.958633,1
996,4577,8018E,Present,"{'latitude': '-37.82953407573397', 'longitude'...",-37.829534,144.969054,1
997,6758,13099W,Present,"{'latitude': '-37.821005322182685', 'longitude...",-37.821005,144.944223,1
998,6284,13446E,Present,"{'latitude': '-37.819218167179116', 'longitude...",-37.819218,144.948627,1


In [8]:
live_df.drop_duplicates("bay_id", inplace=True)
live_df.head()

,bay_id,st_marker_id,status,location,lat,lon,:@computed_region_evbi_jbp8
0,2242,3930N,Unoccupied,"{'latitude': '-37.81219457073095', 'longitude'...",-37.812195,144.954629,1
1,3843,C5566,Unoccupied,"{'latitude': '-37.80641793309145', 'longitude'...",-37.806418,144.953969,1
2,2569,C3128,Present,"{'latitude': '-37.81353464672647', 'longitude'...",-37.813535,144.957970,1
3,4043,C6124,Unoccupied,"{'latitude': '-37.80451697889771', 'longitude'...",-37.804517,144.952376,1
4,4344,C7042,Present,"{'latitude': '-37.80749831220392', 'longitude'...",-37.807498,144.958440,1


In [9]:
new_df = live_df[["bay_id", "lat", "lon"]]
new_df

,bay_id,lat,lon
0,2242,-37.812195,144.954629
1,3843,-37.806418,144.953969
2,2569,-37.813535,144.957970
3,4043,-37.804517,144.952376
4,4344,-37.807498,144.958440
...,...,...,...
995,3316,-37.809834,144.958633
996,4577,-37.829534,144.969054
997,6758,-37.821005,144.944223
998,6284,-37.819218,144.948627


In [15]:
df_merge_col = pd.merge(historic_df, new_df, how='left', on='bay_id')
df_merge_col

,deviceid,arrivaltime,departuretime,durationminutes,streetmarker,signplateid,sign,areaname,streetid,streetname,...,betweenstreet1,betweenstreet2id,betweenstreet2,sideofstreet,sidename,bay_id,inviolation,vehiclepresent,lat,lon
0,17605,2020-01-31T15:08:32.000,2020-01-31T16:22:54.000,74,13762E,550.0,Loading Zone 60mins,Docklands,592,DOEPEL WAY,...,DOCKLANDS DRIVE,1048,NEWQUAY PROMENADE,2,East,6650,0,0,NaN,NaN
1,17585,2020-02-26T05:43:24.000,2020-02-26T06:52:06.000,69,13740E,NaN,NaN,Docklands,1097,PEARL RIVER ROAD,...,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6513,0,1,NaN,NaN
2,17562,2020-01-13T22:55:52.000,2020-01-13T22:56:09.000,1,13711W,NaN,NaN,Docklands,1405,WATTLE ROAD,...,LEVEN AVENUE,590,DOCKLANDS DRIVE,5,West,6483,0,1,NaN,NaN
3,17554,2020-02-06T00:00:00.000,2020-02-06T07:30:00.000,450,13697W,NaN,NaN,Docklands,1043,NAVIGATION DRIVE,...,COLLINS STREET,123,BOURKE STREET,5,West,6435,0,0,NaN,NaN
4,17580,2020-01-08T17:05:43.000,2020-01-08T17:40:29.000,35,13730E,196.0,2P MTR M-SAT 7:30-18:30,Docklands,1097,PEARL RIVER ROAD,...,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6508,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,17599,2020-02-19T10:06:27.000,2020-02-19T10:15:48.000,9,13756E,287.0,1P MTR M-SAT 7:30-18:30,Docklands,1097,PEARL RIVER ROAD,...,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6521,0,1,-37.81261,144.936436
996,17584,2020-02-25T09:48:20.000,2020-02-25T14:13:03.000,265,13738E,196.0,2P MTR M-SAT 7:30-18:30,Docklands,1097,PEARL RIVER ROAD,...,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6512,0,0,NaN,NaN
997,17599,2020-01-22T10:21:58.000,2020-01-22T10:33:06.000,12,13756E,287.0,1P MTR M-SAT 7:30-18:30,Docklands,1097,PEARL RIVER ROAD,...,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6521,0,0,-37.81261,144.936436
998,17642,2020-01-09T11:35:14.000,2020-01-09T11:39:26.000,4,13808N,126.0,1/4P M-SUN 7:30-18:30,Docklands,590,DOCKLANDS DRIVE,...,PEARL RIVER ROAD,1398,WATERFRONT WAY,3,North,6474,0,0,-37.81423,144.938833


In [18]:
df_merge_col = df_merge_col.dropna()
df_merge_col

,deviceid,arrivaltime,departuretime,durationminutes,streetmarker,signplateid,sign,areaname,streetid,streetname,...,betweenstreet1,betweenstreet2id,betweenstreet2,sideofstreet,sidename,bay_id,inviolation,vehiclepresent,lat,lon
8,17538,2020-01-28T08:35:23.000,2020-01-28T08:40:17.000,5,13668E,196.0,2P MTR M-SAT 7:30-18:30,Docklands,826,KARLSRUHE LANE,...,COLLINS STREET,830,KEERA WAY,2,East,6421,0,1,-37.821039,144.945431
9,17579,2020-01-11T12:29:49.000,2020-01-11T13:34:06.000,65,13728E,196.0,2P MTR M-SAT 7:30-18:30,Docklands,1097,PEARL RIVER ROAD,...,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6507,0,0,-37.814241,144.937424
16,17571,2020-03-04T17:04:21.000,2020-03-04T17:05:28.000,1,13720E,196.0,2P MTR M-SAT 7:30-18:30,Docklands,1097,PEARL RIVER ROAD,...,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6503,0,0,-37.814433,144.937540
17,17586,2020-02-20T12:35:37.000,2020-02-20T13:49:28.000,74,13742E,196.0,2P MTR M-SAT 7:30-18:30,Docklands,1097,PEARL RIVER ROAD,...,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6514,0,1,-37.813056,144.936714
20,17591,2020-01-13T15:22:38.000,2020-01-13T15:26:22.000,4,13748E,287.0,1P MTR M-SAT 7:30-18:30,Docklands,1097,PEARL RIVER ROAD,...,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6517,0,1,-37.812812,144.936558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,17541,2020-01-31T08:23:49.000,2020-01-31T08:39:08.000,16,13671W,196.0,2P MTR M-SAT 7:30-18:30,Docklands,826,KARLSRUHE LANE,...,COLLINS STREET,830,KEERA WAY,5,West,6419,0,1,-37.820998,144.945324
993,17568,2020-02-01T13:50:41.000,2020-02-01T14:33:58.000,43,13717W,463.0,3P MTR M-SAT 7.30 - 18.30,Docklands,1097,PEARL RIVER ROAD,...,BAUDIN LANE,590,DOCKLANDS DRIVE,5,West,6496,0,1,-37.815765,144.938012
995,17599,2020-02-19T10:06:27.000,2020-02-19T10:15:48.000,9,13756E,287.0,1P MTR M-SAT 7:30-18:30,Docklands,1097,PEARL RIVER ROAD,...,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6521,0,1,-37.812610,144.936436
997,17599,2020-01-22T10:21:58.000,2020-01-22T10:33:06.000,12,13756E,287.0,1P MTR M-SAT 7:30-18:30,Docklands,1097,PEARL RIVER ROAD,...,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6521,0,0,-37.812610,144.936436
